In [7]:
import gurobipy as gp
from gurobipy import GRB

# Define the number of orders, plants, and time periods
n = 3  # Example value, replace with actual number of orders
m = 2   # Example value, replace with actual number of plants
T = 15   # Example value, replace with actual number of time periods (cosider this as T+1)

# Parameters (these should be initialized with actual data)
wi = [10, 12, 10]  # Example values for profit weights for orders
dik = [[5, 6], [6, 4], [6, 4]]  # Example values for transportation costs from customer to plant
d0ik = [[4, 4], [3, 5], [5, 4]]  # Example values for return transportation costs
vk = [1, 1]  # Example values for initial number of vehicles at each plant
Ck = [3, 3]  # Example values for production capacity of each plant
lik = [3, 7, 11, 4, 5, 10]  # Example values for parameters associated with the orders
fik = [10, 12, 13, 11, 14, 13] #Return [10, 11], [12, 14], [13, 13]

# Initialize model
model = gp.Model("Production_and_Delivery_Scheduling")

# Decision variables
x = model.addVars(n, m, vtype=GRB.BINARY, name="x")
z = model.addVars(n, m, vtype=GRB.BINARY, name="z")
y = model.addVars(T, m, vtype=GRB.INTEGER, name="y")

# Objective function
model.setObjective(
    gp.quicksum(wi[i] * x[i, k] for i in range(n) for k in range(m)) -
    gp.quicksum(dik[i][k] * x[i, k] for i in range(n) for k in range(m)) -
    gp.quicksum(d0ik[i][k] * z[i, k] for i in range(n) for k in range(m)),
    GRB.MAXIMIZE
)

# Constraints
# (1) An order can be processed in at most one plant
model.addConstrs((gp.quicksum(x[i, k] for k in range(m)) <= 1 for i in range(n)), "OrderProcessing")

# (2) After an order is served, the vehicle must return to a production plant
model.addConstrs((gp.quicksum(x[i, k] for k in range(m)) == gp.quicksum(z[i, k] for k in range(m)) for i in range(n)), "VehicleReturn")

# (3) Initial number of vehicles at each plant
model.addConstrs((y[0, k] == vk[k] for k in range(m)), "InitialVehicles")

# (4) Number of available vehicles at each plant in every instant t
model.addConstrs((
    y[t, k] == y[0, k] - gp.quicksum(x[i, k] for i in range(n) if lik[i] < t) + gp.quicksum(z[i, k] for i in range(n) if fik[i] <= t)
    for t in range(1,T) for k in range(m)), "AvailableVehicles")

# (5) An order i may be processed in plant k if there are available vehicles in instant lik
model.addConstrs((
    gp.quicksum(x[i, k] for i in range(n) if lik[i] == t) <= y[t, k]
    for t in range(1,T) for k in range(m)), "OrderProcessingAtPlant")

# (6) No more than Ck orders are assigned to any time period in any production plant
model.addConstrs((
    gp.quicksum(x[i, k] for i in range(n) if lik[i] == t) <= Ck[k]
    for t in range(1,T) for k in range(m)), "ProductionCapacity")

# Optimize model
model.optimize()

# Print the solution
for v in model.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective Value: {model.objVal}')

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 92 rows, 42 columns and 178 nonzeros
Model fingerprint: 0x8481b4b0
Variable types: 0 continuous, 42 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]


Presolve removed 82 rows and 30 columns
Presolve time: 0.00s
Presolved: 10 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 3.0000000
Found heuristic solution: objective 8.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         8.00000    8.00000  0.00%     -    0s

Explored 1 nodes (3 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 8 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%
x[0,0]: 1.0
x[0,1]: 0.0
x[1,0]: 0.0
x[1,1]: 1.0
x[2,0]: 0.0
x[2,1]: 1.0
z[0,0]: -0.0
z[0,1]: 1.0
z[1,0]: 1.0
z[1,1]: 0.0
z[2,0]: -0.0
z[2,1]: 1.0
y[0,0]: 1.0
y[0,1]: 1.0
y[1,0]: 1.0